In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

web_driver = webdriver.Chrome()

In [25]:
ress_parts = []
ress_models = []

model_id = 0

while True:
    web_driver.get("https://toyota.jp/service/estimate/options")
    time.sleep(5)

    elms_cars = web_driver.find_elements(By.CLASS_NAME, "myt-cars-module")
    if len(elms_cars)<=model_id:
        break
    car_data = elms_cars[model_id].text
    car_data_list = car_data.split("\n")
    res_car = {
        "model_id": model_id,
        "model_name": car_data_list[0],
        "price_low": car_data_list[1].replace("円（税込）", "").replace("～", "").replace(",", ""),
        "price_high": (
            car_data_list[2].replace("円（税込）", "").replace("～", "").replace(",", "") if len(car_data_list)==3 
            else car_data_list[1].replace("円（税込）", "").replace("～", "").replace(",", "")
        ),
        "img_url": elms_cars[model_id].find_element(By.TAG_NAME, "img").get_attribute(name="src")
    }

    elms_cars[model_id].click()
    ress_models.append(res_car)

    time.sleep(5)

    elms = web_driver.find_elements(By.CLASS_NAME, "myt-estimate-sim-btn-inner")
    elms[1].click()

    time.sleep(10)

    elms = web_driver.find_elements(By.ID, "optionList")
    elms_parts = elms[0].find_elements(By.CLASS_NAME, "myt-estimate-sim-option-unit-detail")
    elms_parts_img = elms[0].find_elements(By.CLASS_NAME, "myt-estimate-sim-option-unit-thumb")
    for elm_parts, elm_parts_img in zip (elms_parts, elms_parts_img):
        res_text = elm_parts.text.split("\n")
        res = {
            "model_id": model_id,
            "name": res_text[0],
            "type": res_text[1],
            "price": res_text[2].replace(",", "").replace("円", ""),
            "img_url": elm_parts_img.find_element(By.TAG_NAME, "img").get_attribute(name="src")
        }
        ress_parts.append(res)
        print(res)
    model_id += 1

{'model_id': 0, 'name': '195/55R16タイヤ＆16×6Jアルミホイール（切削光輝＋ダークグレーメタリック/センターオーナメント付）', 'type': 'メーカーオプション', 'price': '39600', 'img_url': 'https://toyota.jp/tcv/resources/aqua/002_p_006/images/ex/10l.jpg'}
{'model_id': 0, 'name': 'バイカラーLEDフォグランプ（切り替え式）\u3000パノラミックビューモニター付車、LEDフロントフォグランプ付車', 'type': '販売店オプション', 'price': '49500', 'img_url': 'https://toyota.jp/tcv/resources/aqua/002_p_006/images/ex/dcsu.jpg'}
{'model_id': 0, 'name': 'サイドバイザー（RVワイド）', 'type': '販売店オプション', 'price': '13200', 'img_url': 'https://toyota.jp/tcv/resources/aqua/002_p_006/images/ex/k0qb.jpg'}
{'model_id': 0, 'name': 'GRカーボンナンバーフレーム\u3000フロント用', 'type': '販売店オプション', 'price': '19800', 'img_url': 'https://toyota.jp/tcv/resources/aqua/002_p_006/images/ex/vac6.jpg'}
{'model_id': 0, 'name': 'GRカーボンナンバーフレーム\u3000リヤ用', 'type': '販売店オプション', 'price': '19800', 'img_url': 'https://toyota.jp/tcv/resources/aqua/002_p_006/images/ex/vac6.jpg'}
{'model_id': 0, 'name': 'MODELLISTAエアロパーツセット', 'type': '販売店オプション', 'price': '122100', 'img_url'

In [26]:
pd.DataFrame(ress_parts).to_csv("./exterior_parts.csv")

In [27]:
pd.DataFrame(ress_models).to_csv("./models.csv")